<h1 align='center'> Análise Vibracional de um Modelo Quarter Car </h1>

In [1]:
#Importando bibliotecas:
import numpy as np
from sympy import *
from sympy.integrals.heurisch import heurisch
import warnings
import matplotlib.pyplot as plt

#Exibição dos gráficos em no notebook:
%matplotlib inline

'''
#Exibição dos gráficos em janela externa:
%matplotlib qt
plt.rcParams['lines.linewidth'] = 3
plt.rcParams['font.size'] = 20
'''

# Inicializações importantes:
warnings.filterwarnings("ignore")
init_printing()

# Função de arredondar elemento da matriz:
def arredNUM(matrix):
    for a in preorder_traversal(matrix):
        if isinstance(a, Float):
            matrix = matrix.subs(a, round(a, 4))
    return matrix

<div align="center">
    <img src="images/hcpmodel1.png">
    <img src="images/hcpmodel2.png">
</div>

<h5 align='right'> (Jazar, 2017) - 3rd ed </h5>

$$[M]\ddot{x} - [C]\dot{x} + [K]x = [F]$$

<h5 align='right'> Jazar - Eq. (11.46) </h5>

### Parâmetros do sistema:

In [2]:
# Unidades no S.I [m, Kg, N/m, Kg*m², (N*s)/m]:
# GDL não suspenso dianteiro:
a1 = 1.4
m1 = 53
kt1 = 200000

# GDL não suspenso traseiro:
a2 = 1.47
m2 = (152/2)
kt2 = 200000

# GDL suspenso:
Iy = (1100/2)
ms = (840/2)
k1 = 10000
k2 = 13000
c1 = 1800
c2 = 2000

### Excitação externa por perfil de pista:

##### Entrada Harmônica:

In [7]:


# Tempo de amostragem:
ts = 1e-2

# Tempo de simulação:
tf = 80

# Velocidade do carro em Km/h:
v_car = 18

# Amplitude em mm:
amplitude = 250

# Espaçamento em m:
esp = 100

# Offset em mm:
offset = 0

# Função de entrada de deslocamento, em mm:
x = Symbol('x')
freq = round(((v_car*2*np.pi)/(3.6*esp)), 4)
print(f'freq = {freq}')
# Algum bug na heurística impede a resolução com qualquer valor de frequência (será substituído mais adiante no código)
G = Symbol('\omega')
ptt = Symbol('\phi_t')
ptd = Symbol('\phi_d')

phi_t = round(((a1+a2)/v_car), 4)
phi_d = 0

print(f'phi_t = {phi_t}')

ent_t = (((amplitude*sin((x*G)+ptt))+offset)*1e-3)
ent_d = (((amplitude*sin((x*G)+ptd))+offset)*1e-3)

# Criação do vetor de tempo:
tfa = 500
if (tf > tfa):
    tfa = tf
time = np.arange(0, tfa, ts)

yt = lambdify(x, ent_t.subs((G, freq), (ptt, phi_t)), 'numpy')(time)  # visualização do perfil
yd = lambdify(x, ent_d.subs((G, freq), (ptd, phi_d)), 'numpy')(time)  # visualização do perfil

ent_t
#ent_d

freq = 0.3142
phi_t = 0.1594


SyntaxError: unexpected character after line continuation character (<lambdifygenerated-5>, line 2)

##### Dados empíricos:

In [ ]:


'''
# Leitura do arquivo:
ts = 1e-2
arq = open('data.txt', 'r')
g = []
for linha in arq:
    linha = arq.readline()
    if(linha != ''):
        g.append(float(linha))
arq.close()
def actopos(dados):
    v_ant = 0
    current_t = [0]
    current_pos = [0]
    for i in range(len(dados)):
        current_pos.append(round((current_pos[i]+(v_ant*ts)+(dados[i]*(ts**2))), 4))
        current_t.append(ts*(i+1))
    return current_t, current_pos
time, y = actopos(np.multiply(g, (9.8067*1e3)))
tf = np.max(time)
'''

# INCOMLPETO! - Tem que encontrar uma forma de ajustar uma função que atenda sinais senoidais, como soma de senos e cossenos a partir de uma amostra....

##### Plot da superfície:

In [ ]:
'''

# Plotando os gráficos:
plt.rcParams['figure.figsize'] = (12, 5)
plt.figure(-1)
plt.plot((0, tf), (0, 0), 'gray', linestyle='dashed')
plt.plot(time, (y*1e3), 'k')
plt.ylabel('$y(t)~~[mm]$')
plt.xlabel('$Tempo~~[s]$')
plt.xlim(0, tf)
plt.show()
'''

### Definição das matrizes (Lagrange) e condições iniciais:

##### Massa:

In [ ]:
# Fator de correção para cálculos:
fc = 1e-3

# Matriz das massas (Kg):
M = np.array([
    [ms, 0, 0, 0],
    [0, Iy, 0, 0],
    [0, 0, m1, 0],
    [0, 0, 0, m2]
], dtype=float)*fc

# Matriz de rigidez (N/m):
K = np.array([
    [(k1+k2), ((a2*k2)-(a1*k1)), -k1, -k2],
    [((a2*k2)-(a1*k1)), ((k1*(a1**2))+(k2*(a2**2))), (a1*k1), -(a2*k2)],
    [-k1, (a1*k1), (k1+kt1), 0],
    [-k2, -(a2*k2), 0, (k2+kt2)]
], dtype=float)*fc

# Matriz de amortecimento (N*s/m):
C = np.array([
    [(c1+c2), ((a2*c2)-(a1*c1)), -c1, -c2],
    [((a2*c2)-(a1*c1)), ((c1*(a1**2))+(c2*(a2**2))), (a1*c1), -(a2*c2)],
    [-c1, (a1*c1), c1, 0],
    [-c2, -(a2*c2), 0, c2]
], dtype=float)*fc

# Matriz de F(x) (transposta):
F = np.array([0, 0, (ent_d*kt1), (ent_t*kt2)]).T*fc

# Matriz de posições iniciais (transposta):
X0 = np.array([0, 0, 0, 0], dtype=float).T

# Matriz de velocidades iniciais (transposta):
V0 = np.array([0, 0, 0, 0], dtype=float).T

Matrix(F)

### $$\text{Obtenção de } [M]^{-1/2}:$$

In [ ]:
MM = np.power(M, -1/2)  # Inverso da raíz quadrada

MM[MM == np.inf] = 0  # Transforma os elementos infinitos em zero

Matrix(MM.round(4))

### $$ \text{Cálculo da matriz } \tilde{[K]} = [M^{-1/2}][K][M^{-1/2}]:$$

In [ ]:
K_til = MM@K@MM  # Cálculo da matriz K_til

Matrix(K_til.round(4))

### $$ \text{Cálculo da matriz } \tilde{[C]} = [M^{-1/2}][C][M^{-1/2}]:$$

In [ ]:
try:
    C_til = MM@C@MM  # Cálculo da matriz C_til
except:
    C_til = np.zeros((len(M), len(M)))

Matrix(C_til.round(4))

### $$\text{Obtendo os autovalores (} \lambda \text{) e autovetores normalizados (} P \text{) de }\tilde{[K]}:$$

In [ ]:
# Obtenção dos autovalores e autovetores normalizados
lamb, P = np.linalg.eig(K_til)

##### $$\text{Matriz de autovalores }[\lambda]:$$

In [ ]:
Matrix(lamb.round(4))

##### $$\text{Matriz de autovetores normalizados }[P]:$$

In [ ]:
Matrix(P.round(4))

### Cálculo das frequências naturais: $$[\omega_n] = \left[ \sqrt{\lambda} \right]$$

In [ ]:
omega_n = np.sqrt(lamb)  # Calcula a matriz de freqências naturais, em rad/s

Matrix(omega_n.round(4))

### Mode Shapes: $$[u_i] = [M^{-1/2}][P_i]$$

In [ ]:
U = (MM@P)  # Modos de vibrar: cada linha é um GDL e cada coluna representa um modo de vibrar

Matrix(U.round(4))

In [ ]:
horizontal = False  # Define o sentido de variação dos GDL's

# Plotando os gráficos:
plt.rcParams['figure.figsize'] = (12, 10)
plt.figure(0)

space = (np.max(np.abs(U))*2.5)

padding = []
for i in range(len(U)+1):
    padding.append(space*i)

vib = []
for i in range(len(U)):
    vib.append([(space*(i+1))])

for i in range(len(vib)):
    vib[i] = np.concatenate((vib[i], (vib[i][0]+U.T[i])))

if (horizontal):
    for i in range(len(U)):
        plt.plot(vib[i], padding, 'k')
        plt.plot((vib[i][0], vib[i][0]), (0, (space*len(U))),
                 'gray', linestyle='dashed')
    plt.xticks(padding[1:], range(1, len(U)+1))
    labels = []
    for i in range(1, len(U)+1):
        labels.append(f'$y_{i}$')
    plt.yticks(padding[1:], labels)
else:
    for i in range(len(U)):
        plt.plot(padding, vib[i], 'k')
        plt.plot((0, (space*len(U))),
                 (vib[i][0], vib[i][0]), 'gray', linestyle='dashed')
    plt.yticks(padding[1:], range(1, len(U)+1))
    labels = ['$y_s$', '$\\theta$', '$y_1$', '$y_2$']
    plt.xticks(padding[1:], labels)

plt.title('')
plt.show()

### $$\text{Cálculo da matriz } [S] = [M^{-1/2}][P]$$

In [ ]:
S = MM@P  # Cálcula a matriz S

Matrix(S.round(4))

### $$\text{Cálculo da matriz } [\Lambda] = [P^T][\tilde{K}][P]$$

In [ ]:
# Calcula a matriz LAMBDA, que deve ser igual à matriz diagonal contendo os autovalores
LAMB = P.T@K_til@P

Matrix(LAMB.round(4))

### $$\text{Cálculo da matriz } [\Gamma] = [P^T][\tilde{C}][P]$$

In [ ]:
GAM = P.T@C_til@P  # Calcula a matriz GAMMA, que deve ser igual à matriz diagonal contendo os coeficientes de amortecimento em r -> 2*zeta*omega

Matrix(GAM.round(4))

### $$\text{Determinando os coeficientes de amortecimento } \zeta_i:$$

In [ ]:
try:
    Matrix(Z.round(4))
except:
    Z = np.zeros(len(M))
    for i in range(len(omega_n)):
        Z[i] = (GAM[i][i]/(2*np.sqrt(LAMB[i][i])))
Matrix(Z.round(4))

### Cálculo das frequências amortecidas: $$[\omega_d] = \left[ \omega_n\sqrt{1-\zeta^2} \right]$$

In [ ]:
# Calcula a matriz de freqências amortecidas, em rad/s
omega_d = omega_n*np.sqrt(1-np.power(Z, 2))

Matrix(omega_d.round(4))

### $$\text{Cálculo da matriz } [\Pi] = [P^T][M^{-1/2}][F]$$

In [ ]:
# Calcula a matriz PI, que deve ser igual à matriz coluna, contendo as componentes da força aplicada, distribuida nos GDL's
PI = Matrix(P.T@MM@F)

Matrix(arredNUM(PI))

### Transformações de coordenadas das condições iniciais: $$[X_0], [V_0] ~~\rightarrow~~ [R_o], [\dot{R_0}]$$

##### $$\text{Matriz de posição }[X_0] \rightarrow [R_0]:$$

In [ ]:
R0 = np.linalg.inv(S)@X0  # Conversão de coordenadas para posição

Matrix(R0.round(4))

##### $$\text{Matriz de velocidade }[V_0] \rightarrow [\dot{R_0}]:$$

In [ ]:
RD0 = np.linalg.inv(S)@V0  # Conversão de coordenadas para velocidade

Matrix(RD0.round(4))

### Obter soluções modais desacopladas do tipo: $$r_i(t) = A_ie^{-\zeta_i\omega_it}sen(\omega_{d_i}t+\phi_i)+\frac{1}{\omega_{d_i}}e^{-\zeta_i\omega_it}\int_0^t \left( f_i(\tau)e^{\zeta_i\omega_i\tau}sen(\omega_{d_i}(t-\tau)) \right)d\tau$$

### $$\text{Cálculo de } A_i = \sqrt{\frac{(r_{0_i}\omega_{d_i})^2+(\dot{r_{0_i}}+\zeta_i\omega_ir_{0_i})^2}{\omega_{d_i}^2}}:$$

In [ ]:
A = np.sqrt((np.power((R0*omega_n), 2)+np.power((RD0+(Z*omega_n*R0)), 2))/np.power(omega_d, 2))  # Cálculo da matriz de amplitudes

Matrix(A.round(4))

### $$\text{Cálculo de } \phi_i = tan^{-1}\left( \frac{r_{o_i}\omega_{d_i}}{\dot{r_0}_i+\zeta_i\omega_ir_{0_i}} \right):$$

In [ ]:
# Cálculo da matriz de fases, em radianos
ph = np.arctan((R0*omega_n)/(RD0+(Z*omega_n*R0)))

PHI = []

# Ajuste para evitar erros de aproximação do ponto flutuante e indefinições de atan():
for i in range(len(ph)):
    if (RD0[i]+(Z[i]*omega_n[i]*R0[i])):
        if ((ph[i]/np.pi) in (0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0)):
            PHI.append(nsimplify(round((ph[i]/np.pi), 4)*pi))
        else:
            PHI.append(round(ph[i], 4))
    elif (R0[i]*omega_n[i]):
        PHI.append(pi/2)
    else:
        PHI.append(0)

Matrix(PHI)

### Montando as expressões:

In [ ]:
# Definição da forma geral da solução:
a = Symbol('A')
omgn = Symbol('\omega_n')
omgd = Symbol('\omega_d')
phi = Symbol('\phi')
f = Function('f')(x)
z = Symbol('\zeta')
t = Symbol('t')

# Resposta amortecida:
damped = (a*exp(-z*omgn*t)*sin((omgd*t)+phi))

# Método Heurístico para resolução da integral indefinida e teorema fundamental do cálculo:


def itg(fun, a, b):
    expression = heurisch(fun, x)
    return simplify(expression.subs(x, b) - expression.subs(x, a))

# Resposta forçada:


def forced(function=None):
    if (function == None):
        return ((1/omgd)*exp(-z*omgn*t)*integrate((f*exp(z*omgn*x)*sin(omgd*t-omgd*x)), (x, 0, t)))
    else:
        return ((1/omgd)*exp(-z*omgn*t)*itg((function*exp(z*omgn*x)*sin(omgd*t-omgd*x)), 0, t))


(damped+forced())

In [ ]:
# Substituição das respectivas constantes em suas iterações:
RT = []
for i in range(len(omega_n)):
    ri = (damped+forced(PI[i])).subs([(a, A[i]), (omgn, omega_n[i]),
                                      (phi, PHI[i]), (omgd, omega_d[i]), (z, Z[i])]).doit()
    RT.append(trigsimp(arredNUM(ri)))  # Simplificação trigonométrica

# RT = trigsimp(arredNUM(Matrix(RT).subs(G, freq))) # Correção de bug no método heurístico

RT = arredNUM(simplify(trigsimp(Matrix(RT).subs((G, freq), (ptt, phi_t), (ptd, phi_d)))))

Matrix(RT)

### Tempos de acomodação (critério 2%): $$ST = \frac{4}{\zeta\omega_n}$$

In [ ]:
ultim_time = 0
for i in range(len(omega_n)):
    if (Z[i]):
        current = (4/(Z[i]*omega_n[i]))
        if (current > ultim_time):
            ultim_time = np.ceil(current*1.5)
        print(f'ST{i+1} = {round(current, 2)} s')
    else:
        print(f'ST{i+1} = {np.Infinity}')

### Respostas desacopladas:

In [ ]:
# Criando matriz de posição em função do tempo:
R = []
for i in RT:
    fx = lambdify(t, i, 'numpy')
    R.append(fx(time))

# Plotando os gráficos:
plt.rcParams['figure.figsize'] = (12, 5)
plt.figure(1)
labels = ['$r_u$', '$r_s$']
for i in range(len(R)):
    plt.plot(time, (R[i]*1e3), label=labels[i])
plt.ylabel('r')
plt.legend()
plt.xlabel('Tempo [s]')
plt.title('')
plt.xlim(0, tf)
plt.grid()
plt.show()

### Retornando ao domínio físico:

In [ ]:
XT = arredNUM(trigsimp(Matrix(S@RT)))

Matrix(XT)

### Resposta temporal:

In [ ]:
# Criando matriz de posição em função do tempo:
X = []
for i in XT:
    fx = lambdify(t, i, 'numpy')
    X.append(fx(time))

# Plotando os gráficos:
plt.rcParams['figure.figsize'] = (12, 5)
plt.figure(2)
plt.suptitle('')
plt.subplots(len(X), sharex=True)
labels = ['$x_u(t)~~[mm]$', '$x_s(t)~~[mm]$']
for i in range(len(X)):
    plt.subplot(len(X), 1, (i+1))
    plt.plot(time, (X[i]*1e3))
    plt.ylabel(labels[i])
    # plt.legend()
    plt.grid()
plt.xlim(0, tf)
plt.xlabel('Tempo [s]')
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = (12, 5)
plt.figure(len(omega_n)+2)
labels = ['$x_u(t)$', '$x_s(t)$']
for i in range(len(X)):
    plt.plot(time, (X[i]*1e3), label=labels[i])
plt.ylabel('x [mm]')
plt.legend()
plt.xlabel('Tempo [s]')
plt.title('')
plt.xlim(0, tf)
plt.grid()
plt.show()

### Steady State:

In [ ]:
XSS = XT
for a in preorder_traversal(XSS):
    if isinstance(a, type(exp(t))):
        XSS = XSS.subs(a, 0)

Matrix(XSS)

In [ ]:
# Criando matriz de posição em função do tempo:

def const_function(time, cte):
    return ((time*0)+cte)


XSST = []
for i in XSS:
    fx = lambdify(t, i, 'numpy')
    css = fx(tf+time)
    if (isinstance(css, int)):
        for a in preorder_traversal(i):
            XSST.append(const_function(tf+time, a))
    else:
        XSST.append(css)

# Plotando os gráficos:
plt.rcParams['figure.figsize'] = (12, 5)
plt.figure(3)
labels = ['$Xss_u(t)$', '$Xss_s(t)$']
for i in range(len(XSST)):
    plt.plot((tf+time), (XSST[i]*1e3), label=labels[i])
plt.ylabel('X [mm]')
plt.legend()
plt.xlabel('Tempo [s]')
plt.title('')
plt.xlim(tf, (2*tf))
plt.grid()
plt.show()

### Frequências naturais, em Hertz:

In [ ]:
for i in range(len(omega_n)):
    print(f'f{i+1} = {round((omega_n[i]/(2*np.pi)), 4)} Hz')

### Resposta em frequência:

In [ ]:
# Preparando as transformadas de Fourier:
def fft(sig, ts):
    fft = np.fft.fft(sig)

    N = len(sig)

    g = np.fft.fftfreq(N, ts)

    freqs = (g[:(N//2)]*(2*np.pi))
    amplitudes = (np.abs(fft)[:(N//2)]*(1/N))
    phase = np.rad2deg(np.angle(fft)[:(N//2)])

    return freqs, amplitudes, phase


ffts = []
for i in X:
    ffts.append(fft(i, ts))


# Frequências de ressonância
freqres = []
amp = []
for i in ffts:
    j = np.where(i[1] == np.max(i[1]))
    freqres.append(i[0][j][0])
    amp.append(i[1][j][0])

# Limite do eixo X, para ajuste:
xl = (np.max(freqres)/np.pi)

# Pltotando: Amplitude em função de frequência
plt.rcParams['figure.figsize'] = (12, 5)
plt.figure(4)
plt.semilogy()
labels = ['$x_u(j\omega)$', '$x_s(j\omega)$']
for i in range(len(ffts)):
    plt.plot((ffts[i][0]/(2*np.pi)), ffts[i][1], label=labels[i])
    plt.plot([(freqres[i]/(2*np.pi)), (freqres[i]/(2*np.pi))],
             [0, amp[i]], 'gray', linestyle='dashed', label=f'$fr_{i+1}$')
plt.ylabel('Magnitude [dB]')
plt.legend()
plt.xlabel('Frequência [Hz]')
plt.title('')
plt.xlim(0, xl)
plt.grid()
plt.show()

### Frequências de ressonância, em Hertz:

In [ ]:
for i in range(len(freqres)):
    print(f'f{i+1} = {(freqres[i]/(2*np.pi)).round(4)} Hz')